In [2]:
import qiskit
from qiskit import QuantumCircuit, Aer, execute
import qiskit.providers.aer.library
from qiskit.quantum_info import random_statevector
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms.optimizers import COBYLA, ADAM
from qiskit.circuit import ParameterVector
from qiskit.algorithms import VQE

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl

from random import random
import numpy as np
from scipy.optimize import minimize
from tqdm import tqdm
import itertools

In [29]:
def add_layers(circuit, params):
    
    for p in range(0, len(params), 6):
        
        if circuit.num_qubits == 4:

            circuit.rxx(params[p],0, 3)
            circuit.rxx(params[p],1, 2)
            for i in range(4):
                circuit.rz(params[p + 3],i)
            circuit.barrier()

            circuit.ryy(params[p + 1], 0, 1)
            circuit.ryy(params[p + 1], 2, 3)
            for i in range(4):
                circuit.rx(params[p+4],i)
            circuit.barrier()

            circuit.rzz(params[p + 2], 0, 2)
            circuit.rzz(params[p + 2], 1, 3)
            for i in range(4):
                circuit.ry(params[p+5],i)
            circuit.barrier()
        
        elif circuit.num_qubits == 8:
            
            circuit.rxx(params[p], 3, 7)
            circuit.rxx(params[p], 2, 6)
            circuit.rxx(params[p], 0, 1)
            circuit.rxx(params[p], 4, 5)
            for i in range(8):
                circuit.rz(params[p+3],i)
            circuit.barrier()

            circuit.ryy(params[p + 1], 2, 3)
            circuit.ryy(params[p + 1], 6, 7)
            circuit.ryy(params[p + 1], 0, 4)
            circuit.ryy(params[p + 1], 1, 5)
            for i in range(8):
                circuit.rx(params[p+4],i)
            circuit.barrier()

            circuit.rzz(params[p + 2], 1, 2)
            circuit.rzz(params[p + 2], 4, 7)
            circuit.rzz(params[p + 2], 3, 6)
            circuit.rzz(params[p + 2], 0, 5)
            for i in range(8):
                circuit.ry(params[p+5],i)
            circuit.barrier()
        
        elif circuit.num_qubits == 16:

            circuit.rxx(params[p], 14, 15)
            circuit.rxx(params[p], 6, 7)
            circuit.rxx(params[p], 0, 8)
            circuit.rxx(params[p], 1, 9)
            circuit.rxx(params[p], 13, 5)
            circuit.rxx(params[p], 12, 4)
            circuit.rxx(params[p], 3, 2)
            circuit.rxx(params[p], 11, 10)
            for i in range(16):
                circuit.rz(params[p + 3], i)
            circuit.barrier()

            circuit.ryy(params[p + 1], 15, 7)
            circuit.ryy(params[p + 1], 14, 6)
            circuit.ryy(params[p + 1], 8, 9)
            circuit.ryy(params[p + 1], 0, 1)
            circuit.ryy(params[p + 1], 5, 4)
            circuit.ryy(params[p + 1], 13, 2)
            circuit.ryy(params[p + 1], 2, 10)
            circuit.ryy(params[p + 1], 3, 11)
            for i in range(16):
                circuit.rx(params[p + 4], i)
            circuit.barrier()

            circuit.rzz(params[p + 2], 15, 12)
            circuit.rzz(params[p + 2], 6, 5)
            circuit.rzz(params[p + 2], 1, 2)
            circuit.rzz(params[p + 2], 8, 11)
            circuit.rzz(params[p + 2], 7, 0)
            circuit.rzz(params[p + 2], 9, 14)
            circuit.rzz(params[p + 2], 3, 4)
            circuit.rzz(params[p + 2], 10, 13)
            for i in range(16):
                circuit.ry(params[p + 5], i)
            circuit.barrier()
        
    return circuit

In [30]:
def create_state_vector(num_qubits, params):
    circuit = QuantumCircuit(num_qubits)
    add_layers(circuit, params)
    backend = Aer.get_backend('statevector_simulator')    
    job = execute(circuit, backend)
    result = job.result()
    statevector = result.get_statevector()
    return statevector

def create_circuit(num_qubits, params):
    circuit = QuantumCircuit(num_qubits)
    add_layers(circuit, params)
    return circuit

def create_random_param(depth):
    params = []
    for _ in range(depth * 6):
        params.append(random() * 2 * np.pi)
    return params

In [23]:
def expectation_value(counts, shots, interaction):
    
    def xx_bitstring(string):
        energy = 0
        energy -= (1 if string[0] == string[3] else -1)
        energy -= (1 if string[1] == string[2] else -1)
        return energy
    
    def yy_bitstring(string):
        energy = 0
        energy -= (1 if string[0] == string[1] else -1)
        energy -= (1 if string[2] == string[3] else -1)
        return energy

    def zz_bitstring(string):
        energy = 0
        energy -= (1 if string[0] == string[2] else -1)
        energy -= (1 if string[1] == string[3] else -1)
        return energy
    
    energy = 0
    for k,v in counts.items():
        if interaction == 'xx':    
            energy += xx_bitstring(k) * (v / shots)
        elif interaction == 'yy':
            energy += yy_bitstring(k) * (v / shots)
        elif interaction == 'zz':
            energy += zz_bitstring(k) * (v / shots)        
    return energy

In [24]:
"""
Cost function parameters
    @ params = variational parameters of the circuit
    @ num_qubits = {4,8,12}
    @ shots = sampling size
"""
def cost_function(params, num_qubits, shots):

    backend = Aer.get_backend('qasm_simulator')
    
    # Z basis measurement
    circuit = create_circuit(num_qubits, params)  
    circuit.measure_all()
    job = execute(circuit, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(circuit)
    zz_exp = expectation_value(counts, shots, 'zz')

    # X basis measurement
    circuit = create_circuit(num_qubits, params)  
    for i in range(num_qubits):
        circuit.h(i)
    circuit.measure_all()
    job = execute(circuit, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(circuit)
    xx_exp = expectation_value(counts, shots, 'xx')
    
    # Y Basis measurement
    circuit = create_circuit(num_qubits, params)  
    for i in range(num_qubits):
        circuit.sdg(i)
        circuit.h(i)
    circuit.measure_all()
    job = execute(circuit, backend, shots = shots)
    result = job.result()
    counts = result.get_counts(circuit)
    yy_exp = expectation_value(counts, shots, 'yy')

    return (zz_exp + xx_exp + yy_exp) * -1